In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
import statistics as stat

In [2]:
data = pd.read_csv('scdb-20rfq-30cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [3]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

1538

In [4]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

bid_count = 0
bid_ids = []
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_ids.append(data_unique_ids.iloc[i]['Tx_id'])
        bid_count += 1   


print(f"#BIDs =  {bid_count}")

#BIDs =  500


In [5]:
df_test = grouped_data.get_group(bid_ids[8])
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
3182,received_tx,109,BID,e0dd8ca51b5e70c2e43ea0852e651d29e6c4219e58bbe8...,None,1.669905e+09
3183,before_tendermint,120,BID,e0dd8ca51b5e70c2e43ea0852e651d29e6c4219e58bbe8...,None,1.669905e+09
3184,check_tx,131,BID,e0dd8ca51b5e70c2e43ea0852e651d29e6c4219e58bbe8...,None,1.669905e+09
3185,deliver_tx,146,BID,e0dd8ca51b5e70c2e43ea0852e651d29e6c4219e58bbe8...,None,1.669905e+09
3186,end_block,148,BID,e0dd8ca51b5e70c2e43ea0852e651d29e6c4219e58bbe8...,None,1.669905e+09
3187,commit_tx,191,BID,e0dd8ca51b5e70c2e43ea0852e651d29e6c4219e58bbe8...,None,1.669905e+09


In [6]:
time_all_committed_bids = {}
time_passed_bids = []
validated_time = []
accepted_time = []
committed_time = []

bid_underfit = []
bid_underfit_dict = {}

bid_overfit = []
bid_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "BID" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            bid_underfit.append(temp_list)
            bid_underfit_dict[tx_id[i]] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_bids[tx_id[i]] = actual_time
            #time_passed_bids.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            
            bid_overfit.append(actual_time)
            bid_overfit_dict[tx_id[i]] = temp_list

In [30]:
df = grouped_data.get_group("3066cad3ab9009c84499c002f3b150864fc0b5af46bd4cb1ab5407ab2396a336")
df[df['Event'] == 'received_tx']['Time'].to_list()[0]

106

In [7]:
bid_overfit_dict

{'58faab5aa8e70cc129184d74111db33caecbaa5ab1aeed8f80f3fad86adb9cf0': [108,
  120,
  140,
  335,
  395,
  397,
  439],
 '645474f63ba04a74915b421d26ccfd5c941cec78b0d111f9ea37d0d83557da70': [109,
  118,
  130,
  256,
  316,
  318,
  364],
 '7ef1c9e28a43a33d008f0788a0264d75027b4e0b33252720d0a3c6cd3230d20f': [107,
  118,
  141,
  334,
  394,
  395,
  502],
 '90cd5b8e5a832a8ad6718284102fa698fd981606f27416d56da0f8869885f8f2': [108,
  119,
  142,
  340,
  596,
  598,
  643],
 'abc9ec57d5851a6fbb799b2e9d596d12dfb7aad0ca6f97f141bb71fa32a825b3': [109,
  119,
  142,
  333,
  389,
  390,
  434],
 'ac74ccf068579d3e56d33ca416b47c7e387c4bebef7699f916f3281c6d1fb2bd': [107,
  118,
  143,
  340,
  503,
  504,
  548],
 'b2165ed7d02aa0d3d8127f208490fdf7b76b0c339990945e6a75dcc09278bc77': [107,
  118,
  130,
  180,
  243,
  245,
  288],
 'e00f3d59a263ef4637faed9d7d19e5b6d5b7f33303741a07c6307dfe7b13f351': [107,
  118,
  141,
  333,
  393,
  394,
  437],
 'e2948e4af3247d827c8cb4e5a8f5243cc1187f3905d255beb6c2f6

In [8]:
len(time_passed_bids)

490

In [9]:
len(bid_overfit)

9

In [10]:
stat.mean(time_passed_bids)

91.65510204081633

In [11]:
stat.mean(bid_overfit)

348.22222222222223

In [12]:
# Analysis of CREATE Transaction
bid_mean = "{:.2f}".format(stat.mean(time_passed_bids))
bid_median = stat.median(time_passed_bids)
bid_max = max(time_passed_bids)
bid_min = min(time_passed_bids)
bid_std_dev = "{:.2f}".format(stat.stdev(time_passed_bids))

print(bid_mean)
print(bid_median)
print(bid_max)
print(bid_min)
print(bid_std_dev)
print(f"total bids = {len(time_passed_bids)}")

bid_dict = {
               'min': bid_min,
               'max': bid_max,
               'average': bid_mean,
               'median' : bid_median,
               'standard_deviation' : bid_std_dev,
               '# of txs' : len(time_passed_bids)}

91.66
85.0
140
78
14.16
total bids = 490


In [13]:
results = pd.DataFrame(bid_dict, index = ['BID'])
results.to_csv('bid_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
BID,78,140,91.66,85.0,14.16,490
